# Audio Features

In [1]:
#import necessary libraries
from mido import MidiFile, MidiTrack, Message
from mido.midifiles.meta import MetaMessage
from scipy.spatial import distance
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
import mido
import os
import math
import numpy as np
import pandas as pd
import math
from sklearn import metrics
import statistics
import random
import librosa
import itertools

<h3>Data Collection from MIDI files

In [2]:
#read midis for jigs
os.getcwd()
files_j = []
os.chdir("/home/girija/Documents/Project/midifiles/")
for root, dirs, files in os.walk("."):
    for file in files:
        files_j.append(mido.MidiFile(file))
        
#read midis for reels
os.getcwd()
files_r = []
os.chdir("/home/girija/Documents/Project/reels/")
for root, dirs, files in os.walk("."):
    for file in files:
        files_r.append(mido.MidiFile(file))
        
#read midis for polkas
os.getcwd()
files_p = []
os.chdir("/home/girija/Documents/Project/polkas/")
for root, dirs, files in os.walk("."):
    for file in files:
        files_p.append(mido.MidiFile(file))  
        
#read midis for barndances
os.getcwd()
files_b = []
os.chdir("/home/girija/Documents/Project/barndances/")
for root, dirs, files in os.walk("."):
    for file in files:
        files_b.append(mido.MidiFile(file))

<h3>Data Extraction from MIDI files (Pitch/Duration/Note-event type)

In [3]:
#get pitch and duration for jigs
pitch_j = []
duration_j = []
note_j = []
for mid in files_j:
    p = []
    d = []
    n = []
    delta = 0
    for track in mid.tracks:
        for message in track:
            if not isinstance(message, MetaMessage):
                p.append(message.note)
                d.append(message.time)
                n.append(message.type)
    pitch_j.append(p)
    duration_j.append(d)
    note_j.append(n)
    
#get pitch and duration for reels
pitch_r = []
duration_r = []
note_r = []
for mid in files_r:
    p = []
    d = []
    n = []
    delta = 0
    for track in mid.tracks:
        for message in track:
            if not isinstance(message, MetaMessage):
                p.append(message.note)
                d.append(message.time)
                n.append(message.type)
    pitch_r.append(p)
    duration_r.append(d)
    note_r.append(n)
    
#get pitch and duration for polkas
pitch_p = []
duration_p = []
note_p = []
for mid in files_p:
    p = []
    d = []
    n = []
    delta = 0
    for track in mid.tracks:
        for message in track:
            if not isinstance(message, MetaMessage):
                p.append(message.note)
                d.append(message.time)
                n.append(message.type)
    pitch_p.append(p)
    duration_p.append(d)
    note_p.append(n)
    
#get pitch and duration for barndances
pitch_b = []
duration_b = []
note_b = []
for mid in files_b:
    p = []
    d = []
    n = []
    delta = 0
    for track in mid.tracks:
        for message in track:
            if not isinstance(message, MetaMessage):
                p.append(message.note)
                d.append(message.time)
                n.append(message.type)
    pitch_b.append(p)
    duration_b.append(d)
    note_b.append(n)

<h3>Data Transformation by cropping Pitch Values to minimum length

In [4]:
#Find minimum length in jigs.
l = []
idx = []
for mid in pitch_j:
    l.append(len(mid))
minimum1 = min(l)

#Find minimum length in reels
l = []
idx = []
for mid in pitch_r:
    l.append(len(mid))
minimum2 = min(l)

#Find minimum length in polkas
l = []
idx = []
for mid in pitch_p:
    l.append(len(mid))
minimum3 = min(l)

#Find minimum length in barndances
l = []
idx = []
for mid in pitch_b:
    l.append(len(mid))
minimum4 = min(l)

#This will be the final minimum value to which the tunes will be cropped irrespective of their genre.
minimum_pitch = min(minimum1,minimum2,minimum3,minimum4)

In [5]:
#crop the notes to minimum length for jigs, reels, polkas and barndances
input_data_jigs = []
for mid in pitch_j:
    input_data_jigs.append(mid[0:minimum_pitch])
    
input_data_reel = []
for mid in pitch_r:
    input_data_reel.append(mid[0:minimum_pitch])
    
input_data_polkas = []
for mid in pitch_p:
    input_data_polkas.append(mid[0:minimum_pitch])
    
input_data_barndances = []
for mid in pitch_b:
    input_data_barndances.append(mid[0:minimum_pitch])

<h3>K-Means Clustering Algorithm

In [6]:
def KMeansClustering(dataset, k, max_itr):
    #create an empty list of centroids
    centroids = []
    #select first 'k' datapoints as the centroids
    for i in range(k):
        centroids.append(dataset[i])
    
    for itr in range(max_itr):
        clusters = []
        #find distance between a data-point and each centroid
        for point in dataset:
            distances = [np.linalg.norm(np.asarray(point)-np.asarray(centroid)) for centroid in centroids]
            #select centroid at minimum distance
            clusters.append(distances.index(min(distances)))

        #empty dictionary to store data-points as per cluster numbers    
        data = {}
        for i in range(k):
            data[i] = []
        
        #append data-points to the list based on cluster number    
        for i in range(len(dataset)):
            data[clusters[i]].append(dataset[i])
        
        #update centroids based on mean of data-points in the cluster    
        centroids = []
        for point in data:
            centroids.append(np.mean(data[point],axis = 0))
            
    return(clusters)

<h3>Silhouette Analysis

In [7]:
#Evaluation using silhouette analysis
def silhouette_analysis(datasets,k, clusters):
    a = []
    intra_cluster = []
    for i in range(k):
        tmp = []
        #find tunes belonging to a particular cluster
        for j in range(len(datasets)):
            if(i==clusters[j]):
                tmp.append(j)
        intra_cluster.append(tmp)
    
    #find intra-cluster distances of all data-points
    for cluster in intra_cluster:
        for i in range(len(cluster)):
            other_points = cluster[:i]+cluster[i+1:]
            dist = []
            flag = 0
            for point in other_points:
                if(len(other_points)>1):
                    dist.append(np.linalg.norm(np.asarray(datasets[point])-np.asarray(datasets[cluster[i]])))
                else:
                    dist.append(np.mean(datasets[point]))
            a.append(np.mean(dist))
    
    #find inter-cluster distances of all data-points
    inter_clusters = []
    for i in range(len(intra_cluster)):
        inter_clusters.append(list(itertools.chain.from_iterable(intra_cluster[:i]+intra_cluster[i+1:])))
    
    b = []
    for i in range(len(intra_cluster)):
        for j in range(len(inter_clusters)):
            if(i==j):
                for given_point in intra_cluster[i]:
                    dist = []
                    for other_point in inter_clusters[j]:
                        dist.append(np.linalg.norm(np.asarray(datasets[given_point])-np.asarray(datasets[other_point])))
                    b.append(min(dist))
    
    #find silhouette co-efficient of each point
    result = []
    for i in range(len(a)):
        for j in range(len(b)):
            if(i==j):
                res = (b[i]-a[i])/(max(a[i],b[i]))
                if(not math.isnan(res)):
                    result.append(res)
    return(statistics.mean(result))

<h3>Pitch Values

<h3>Approach 1: Considering Pitch Value Only

In [8]:
#Pitch values for Jigs
diff_notes3j = []
for item in input_data_jigs:
    diff = []
    
    for i in range(1,len(item)):
        diff.append(item[i])
    diff_notes3j.append(diff)
    
#Pitch values for reels:
diff_notes3r = []
for item in input_data_reel:
    diff = []
    for i in range(len(item)):
        diff.append(item[i])
    diff_notes3r.append(diff)
    
#Pitch values for polkas:
diff_notes3p= []
for item in input_data_polkas:
    diff = []
    for i in range(len(item)):
        diff.append(item[i])
    diff_notes3p.append(diff)
    
#Pitch values for barndances:
diff_notes3b = []
for item in input_data_barndances:
    diff = []
    for i in range(len(item)):
        diff.append(item[i])
    diff_notes3b.append(diff)

<h3>Result of Approach 1 on K-Means

In [9]:
#create dataset
dataset = input_data_jigs+input_data_reel+input_data_polkas+input_data_barndances

In [10]:
#find result
total = 0
for i in range(1):
    clusters = KMeansClustering(dataset, 4, 100)
    total+=silhouette_analysis(dataset,4, clusters)
print("Average Silhouette Coefficient for Approach 1:",total)

Average Silhouette Coefficient for Approach 1: -0.2473433916318066


<h3>Approach 2: Considering difference with mean value

In [11]:
#Pitch values for jigs considering difference with mean value
diff_notes3j = []
for item in input_data_jigs:
    diff = []
    avg = statistics.mean(item)
    for i in range(len(item)):
        diff.append(item[i]-avg)
    diff_notes3j.append(diff)
    
#Pitch values for reels considering difference with mean value
diff_notes3r = []
for item in input_data_reel:
    diff = []
    avg = statistics.mean(item)
    for i in range(len(item)):
        diff.append(item[i]-avg)
    diff_notes3r.append(diff)
    
#Pitch values for polkas considering difference with mean value
diff_notes3p= []
for item in input_data_polkas:
    diff = []
    avg = statistics.mean(item)
    for i in range(len(item)):
        diff.append(item[i]-avg)
    diff_notes3p.append(diff)
    
#Pitch values for barndances considering difference with mean value
diff_notes3b = []
for item in input_data_barndances:
    diff = []
    avg = statistics.mean(item)
    for i in range(len(item)):
        diff.append(item[i]-avg)
    diff_notes3b.append(diff)

<h3>Result of Approach 2 on K-Means

In [12]:
#create dataset
dataset = diff_notes3b+diff_notes3j+diff_notes3p+diff_notes3r

In [13]:
#find result
total = 0
for i in range(1):
    clusters = KMeansClustering(dataset, 4, 200)
    total+=silhouette_analysis(dataset,4, clusters)
print("Average Silhouette Coefficient for Approach 2:",abs(total))

Average Silhouette Coefficient for Approach 2: 0.3011973720727173


<h3>Approach 3: Considering difference with previous note

In [14]:
#Pitch values for  jigs considering difference with previous note
diff_notes3j = []
for item in input_data_jigs:
    diff = []
    for i in range(1,len(item)):
        diff.append(item[i]-item[i-1])
    diff_notes3j.append(diff)
    
#Pitch values for reels considering difference with previous note
diff_notes3r = []
for item in input_data_reel:
    diff = []
    for i in range(1,len(item)):
        diff.append(item[i]-item[i-1])
    diff_notes3r.append(diff)
    
#Pitch values for polkas considering difference with previous note
diff_notes3p= []
for item in input_data_polkas:
    diff = []
    for i in range(1,len(item)):
        diff.append(item[i]-item[i-1])
    diff_notes3p.append(diff)
    
#Pitch values for barndances considering difference with previous note
diff_notes3b = []
for item in input_data_barndances:
    diff = []
    for i in range(1,len(item)):
        diff.append(item[i]-item[i-1])
    diff_notes3b.append(diff)

<h3>Result of Approach 3 on K-Means

In [15]:
#create dataset
dataset = diff_notes3b+diff_notes3j+diff_notes3p+diff_notes3r

In [16]:
#find result
total = 0
for i in range(1):
    clusters = KMeansClustering(dataset, 4, 200)
    total+=silhouette_analysis(dataset,4, clusters)
print("Average Silhouette Coefficient for Approach 2:",total)

Average Silhouette Coefficient for Approach 2: -0.281491081223425


<h3>Approach 4: Considering difference with first note

In [17]:
#Pitch values for jigs considering difference with first note
diff_notes3j = []
for item in input_data_jigs:
    diff = []
    for i in range(len(item)):
        diff.append(item[i]-item[0])
    diff_notes3j.append(diff)
    
#Pitch values for reels considering difference with first note
diff_notes3r = []
for item in input_data_reel:
    diff = []
    for i in range(len(item)):
        diff.append(item[i]-item[0])
    diff_notes3r.append(diff)
    
#Pitch values for polkas considering difference with first note
diff_notes3p= []
for item in input_data_polkas:
    diff = []
    for i in range(len(item)):
        diff.append(item[i]-item[0])
    diff_notes3p.append(diff)
    
#Pitch values for barndances considering difference with first note
diff_notes3b = []
for item in input_data_barndances:
    diff = []
    for i in range(len(item)):
        diff.append(item[i]-item[0])
    diff_notes3b.append(diff)

<h3>Result of Approach 4 on K-Means

In [18]:
#create dataset
dataset = diff_notes3b+diff_notes3j+diff_notes3p+diff_notes3r

In [19]:
#find result
total = 0
for i in range(1):
    clusters = KMeansClustering(dataset, 4, 200)
    total+=silhouette_analysis(dataset,4, clusters)
print("Average Silhouette Coefficient for Approach 4:",total)

Average Silhouette Coefficient for Approach 4: -0.22791393383813574


<h3>Duration

<h3>Approach 1: Considering original duration values

In [20]:
#find lengths of duration vector in Jigs
l1 = []
for duration in duration_j:
    size = len(duration)
    l1.append(size)
    
#find lengths of duration vector in Reels
l2 = []
for duration in duration_r:
    size = len(duration)
    l2.append(size)
    
#find lengths of duration vector in Polkas
l3 = []
for duration in duration_p:
    size = len(duration)
    l3.append(size)

#find lengths of duration vector in Barndances
l4 = []
for duration in duration_b:
    size = len(duration)
    l4.append(size)
    
#find overall minimum length
minimum_bin = min(min(l1),min(l2),min(l3), min(l4))

In [21]:
#crop the duration to minimum length for jigs, reels, polkas and barndances
duration_jigs = []
for item in duration_j:
    duration_jigs.append(list(item[:minimum_bin]))
    
duration_reels = []
for item in duration_r:
    duration_reels.append(list(item[:minimum_bin]))
    
duration_polkas = []
for item in duration_p:
    duration_polkas.append(list(item[:minimum_bin]))
    
duration_barndances = []
for item in duration_b:
    duration_barndances.append(list(item[:minimum_bin]))

<h3>Result of Approach 1 on K-Means

In [22]:
#create dataset
dataset = duration_jigs+duration_polkas+duration_reels+duration_barndances

In [23]:
#find result
total = 0
for i in range(1):
    clusters = KMeansClustering(dataset, 4, 200)
    total+=silhouette_analysis(dataset,4, clusters)
print("Average Silhouette Coefficient for Approach 1:",total)

Average Silhouette Coefficient for Approach 1: -0.32674595343842366


<h3>Approach 2: Duration in Binary

In [ ]:
#find lengths of duration vector in Jigs
l1 = []
for duration in duration_j:
    size = np.sum(np.asarray(duration))
    l1.append(size)
    
#find lengths of duration vector in Reels
l2 = []
for duration in duration_r:
    size = np.sum(np.asarray(duration))
    l2.append(size)

#find lengths of duration vector in Polkas
l3 = []
for duration in duration_p:
    size = np.sum(np.asarray(duration))
    l3.append(size)

#find lengths of duration vector in Barndances
l4 = []
for duration in duration_b:
    size = np.sum(np.asarray(duration))
    l4.append(size)
    
#find overall minimum length
minimum_bin = min(min(l1),min(l2),min(l3), min(l4))

In [ ]:
#duration(ticks) in binary for jigs
jigs_binary = []
for item in duration_j:
    tmp = np.zeros(minimum_bin)
    i = 0
    for duration in item:
        if(duration==1 and i<minimum_bin):
                tmp[i] = 1
        else:
            tmp[i+1:duration+1] = 0
        i+=duration
    jigs_binary.append(list(tmp))

In [ ]:
#duration(ticks) in binary for reels
reels_binary = []
for item in duration_r:
    tmp = np.zeros(minimum_bin)
    i = 0
    for duration in item:
        if(duration==1 and i<minimum_bin):
                tmp[i] = 1
        else:
            tmp[i+1:duration+1] = 0
        i+=duration
    reels_binary.append(list(tmp))

In [ ]:
#duration(ticks) in binary for polkas
polkas_binary = []
for item in duration_p:
    tmp = np.zeros(minimum_bin)
    i = 0
    for duration in item:
        if(duration==1 and i<minimum_bin):
                tmp[i] = 1
        else:
            tmp[i+1:duration+1] = 0
        i+=duration
    polkas_binary.append(list(tmp))

In [ ]:
#duration(ticks) in binary for barndances
barndances_binary = []
for item in duration_b:
    tmp = np.zeros(minimum_bin)
    i = 0
    for duration in item:
        if(duration==1 and i<minimum_bin):
                tmp[i] = 1
        else:
            tmp[i+1:duration+1] = 0
        i+=duration
    barndances_binary.append(list(tmp))

<h3>Result of Approach 2 on K-Means

In [ ]:
#create dataset
dataset = jigs_binary+polkas_binary+reels_binary+barndances_binary

In [ ]:
#find result
total = 0
for i in range(1):
    clusters = KMeansClustering(dataset, 4, 100)
    total+=silhouette_analysis(dataset,4, clusters)

In [ ]:
print("Average Silhouette Coefficient for Approach 2:",abs(total))

Average Silhouette Coefficient for Approach 2: 0.2095316953639387


# Beats

In [ ]:
#Read all jigs
os.getcwd()
files_j = []
os.chdir("/home/girija/Documents/Project/jigs_wav/")
for root, dirs, files in os.walk("."):
    for file in files:
        files_j.append(file)
        
#find beats for every tune
beats_j = []
for file in files_j:
    y, sr = librosa.load(file)
    tempo, beat = librosa.beat.beat_track(y, sr)
    beats_j.append(beat)

In [ ]:
#Read all reels
os.getcwd()
files_r = []
os.chdir("/home/girija/Documents/Project/reels_wav/")
for root, dirs, files in os.walk("."):
    for file in files:
        files_r.append(file)
        
#find beats for every tune
beats_r = []
for file in files_r:
    y, sr = librosa.load(file)
    tempo, beat = librosa.beat.beat_track(y, sr)
    beats_r.append(beat)

In [ ]:
#Read all polkas
os.getcwd()
files_p = []
os.chdir("/home/girija/Documents/Project/polkas_wav/")
for root, dirs, files in os.walk("."):
    for file in files:
        files_p.append(file)
        
#find beats for every tune
beats_p = []
for file in files_p:
    y, sr = librosa.load(file)
    tempo, beat = librosa.beat.beat_track(y, sr)
    beats_p.append(beat)

In [ ]:
#Read all barndances
os.getcwd()
files_b = []
os.chdir("/home/girija/Documents/Project/barndances_wav/")
for root, dirs, files in os.walk("."):
    for file in files:
        files_b.append(file)
        
#find beats for every tune
beats_b = []
for file in files_b:
    y, sr = librosa.load(file)
    tempo, beat = librosa.beat.beat_track(y, sr)
    beats_b.append(beat)

In [ ]:
#find minimum
#Find minimum length in jigs.
l = []
idx = []
for mid in beats_j:
    l.append(len(mid))
minimum1 = min(l)

#Find minimum length in reels
l = []
idx = []
for mid in beats_r:
    l.append(len(mid))
minimum2 = min(l)
    
#Find minimum length in polkas
l = []
idx = []
for mid in beats_p:
    l.append(len(mid))
minimum3 = min(l)

#Find minimum length in barndances
l = []
idx = []
for mid in beats_b:
    l.append(len(mid))
minimum4 = min(l)

#This will be the final minimum value to which the tunes will be cropped irrespective of their genre.
minimum_beats = min(minimum1,minimum2,minimum3,minimum4)

In [ ]:
#crop to minimum
beats_jigs = []
for item in beats_j:
    beats_jigs.append(list(item[:minimum_beats]))
    
beats_reels = []
for item in beats_r:
    beats_reels.append(list(item[:minimum_beats]))
    
beats_polkas = []
for item in beats_p:
    beats_polkas.append(list(item[:minimum_beats]))
    
beats_barndances = []
for item in beats_b:
    beats_barndances.append(list(item[:minimum_beats]))

<h3>Result of using Vector of Beats on KMeans

In [ ]:
dataset = beats_jigs+beats_reels+beats_polkas+beats_barndances

In [ ]:
total = 0
for i in range(1):
    clusters = KMeansClustering(dataset, 4, 100)
    total+=silhouette_analysis(dataset,4, clusters)

In [ ]:
print("Average Silhouette Coefficient for Beats:",total)